# **Verlet Integration**

<i class="fa fa-home fa-2x"></i><a href="../index.ipynb" style="font-size: 20px"> Go back to index</a>

**Source code:** https://github.com/osscar-org/quantum-mechanics/blob/master/notebook/molecular-dynamics/verlet_integration.ipynb

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

## Goals
* Familiarize yourself with the main features of the Verlet integration scheme employed in the molecular dynamics simulations.
* Know the relationship between position, velocity, and acceleration.
* Acquaint yourself with the numerical technique of finite-differences.
* Appreciate the consequences of the order of the error of the Verlet algorithm.
* Master the mathematics of Verlet integration.

## Background theory
[More on the background theory.](./theory/theory_verlet_integration.ipynb)

## Tasks and exercises

1. Choose a small time step, see how the earth orbits the sun, and check how 
the total energy and angular momentum evolutes with time.

    <details>
    <summary style="color: red">Solution</summary>
    The earth is moving in an elliptical orbit. The earth moves slowly at the end of the ellipse farthest from the sun whilst it moves quickest at the end of the ellipse nearest the sun. However, the total energy 
    and angular momentum remain constant. With a sufficiently small time step the Earth shall remain in this orbit for a very long time.
    </details>
    
2. Does the earth still remain in the same orbit when a large time step is used? And why?

    <details>
    <summary style="color: red">Solution</summary>
    When choosing a large time step, the earth is not keeping on the same 
    ellipse orbital anymore. The ellipse orbitals are shifting for each 
    circle. In the numerical method, we treat the velocities as constant 
    in a short time. But this is not true in the real physical world. When 
    using a large time step, large errors will be introduced into the 
    calculations.
    </details>
    
3. In molecular dynamics simulations, what are the problems associated with choosing (1) a very small timestep, (2) a very large timestep.

    <details>
    <summary style="color: red">Solution</summary>
    In MD simulations, choosing a small time step can improve the 
    accuracy of the results. However, a small time step means taking a 
    longer computing time to obtain the results. Practically, we cannot 
    afford a very small time step for the MD simulations.
    On the other hand, a large time step will introduce large errors. 
    For example, particles can be too close to each other (in the 
    real world, the nuclear-nuclear repulsion energy is considerable at short distances). Hence, it is crucial to choose a reasonable compromise for our 
    time step in MD simulations.
    </details>
    
<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

## Interactive visualization
(be patient, it might take a few seconds to load)

In [ ]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from ipywidgets import Button, IntSlider, Output, VBox, HBox, Layout
from numpy.linalg import norm
import matplotlib.gridspec as gridspec
from datetime import datetime

In [ ]:
def compute_force(r1, r2, m1, m2):
    """"Computer the force for the earth."""
    x = G*m1*m2/np.linalg.norm(r1-r2)**2
    v = (r2-r1)/np.linalg.norm(r2-r1)
    return x*v

In [ ]:
def compute_acceleration(f, m):
    """Compute the acceleration."""
    return f/m

In [ ]:
def verlet_update(r1, r2, v, m1, m2, dt, a):
    """Update the position, velocity and acceleration by the Verlet algorithm."""
    
    rn = 2.0*r1[-1] - r1[-2]  + dt*dt*a
    vn = (rn - r1[-2])/(2*dt)
    
    r1[-2] = r1[-1]
    r1[-1] = rn
    
    f = compute_force(rn, r2, m1, m2)
    a = compute_acceleration(f, m1)
    
    return r1, vn, a

def compute_energies(r1, r2, m1, m2, v):
    """Compute the potential energy."""
    pot = -G*m1*m2/np.linalg.norm(r1-r2)
    kint = 0.5*m1*np.linalg.norm(v)**2
    angmom = np.cross(r1-r2, m1*v)
    
    return pot, kint, angmom

In [ ]:
m1 = 1.0
m2 = 1000.0
G = 0.2
v = [0.0, 3.0]
dt = 0.01

# initial position for the sun and earth, the sun is fix in the centre
r1 = np.array([[10, 0],[10.0, 3.0*dt]])
r2 = np.array([0, 0])

# initial force and acceleration
f = compute_force(r1[-1], r2, m1, m2)
a = compute_acceleration(f, m1)

# the initial total energy
etot0 = -G*m1*m2/np.linalg.norm(r1[0]-r2) + 0.5*m1*np.linalg.norm(v)**2

output = Output()

with output:
    """Output the figures"""
    global fig, ax1, ax2, ax3, sun, earth, origin, avector, fvector, trace, title
    fig = plt.figure(tight_layout=True, figsize=(7.8, 5))
    fig.canvas.header_visible = False
    gs = gridspec.GridSpec(9, 5)
    
    ax1 = fig.add_subplot(gs[:, 0:3])
    ax2 = fig.add_subplot(gs[0:4, 3:5])
    ax3 = fig.add_subplot(gs[5:9, 3:5])

    sun = ax1.scatter([0.0],[0.0], s=23**2, marker="o", linewidth=0, color='gold', label = 'Sun')
    earth, = ax1.plot([10.0],[0.0], 'r.', alpha = 1.0, markersize=15, label = 'Earth')
    origin = ax1.plot([10.0],[0.0], 'kx', alpha = 1.0, markersize=15, label = 'Origin point')
    avector = ax1.quiver([10.0], [0], [0], [0], color = ['blue'], units='inches', scale = 10.0, label="a")
    fvector = ax1.quiver([10.0], [0], [0], [0], color = ['gold'], units='inches', scale = 10.0, label="v")
    trace, = ax1.plot([10.0], [0.0], "k", linewidth=0.5)
    title = ax1.set_title("")

    ax1.legend(loc=2, fontsize=10)

    ax1.set_xlim(-11.0, 11.0)
    ax1.set_ylim(-11.0, 11.0)
    ax1.grid()
    
    lpot, = ax2.plot([], [], 'k-', label='pot')
    lkint, = ax2.plot([], [], 'c-', label='kint')
    ltot, = ax2.plot([],[], 'r-', label='tot')
    ax2.axhline(etot0, color='blue', ls='--', linewidth=0.5)
    
    ax2.set_xlim(0, 20.0)
    ax2.set_ylim(-95, 60)
    ax2.legend(loc=3, ncol=3)
    ax2.set_title("Energies as a function of time", fontsize = 10)
    
    lang, = ax3.plot([],[], 'r-')
    ax3.axhline(30.0, color='blue', ls='--', linewidth=0.5)

    ax3.set_title("Angular momentum as a function of time", fontsize = 10)
    ax3.set_xlabel("Time")
    ax3.set_xlim(0, 20.0)
    ax3.set_ylim(20, 40)


xt = [];
yt = [];

lim1 = 0;
lim2 = 20.0;

t0 = 0.0;
te = [];
epot = [];
ekint = [];
etot = [];
am = [];

def init():
    return (earth, avector, fvector, trace)

    
def update(i):
    """the function to update the frame"""
    global r1, v, a, xt, yt, epot, ekint, etot, te, t0, lim1, lim2, am
    current_time = datetime.now()
    delta_time=(current_time-init_time).total_seconds()
    if (delta_time>=60.0):
        button_pause.description = "Play"
        anim.event_source.stop()
        return 
    r1, v, a = verlet_update(r1, r2, v, m1, m2, dt, a)
    e1, e2, e3 = compute_energies(r1[-1,:], r2, m1, m2, v)

    earth.set_data([r1[-1, 0]], [r1[-1, 1]])
    xt.append(r1[-1, 0])
    yt.append(r1[-1, 1])
    
    epot.append(e1)
    ekint.append(e2)
    etot.append(e1+e2)
    am.append(e3)
    
    t0 = t0 + dt
    te.append(t0)
    
    if te[-1] > 20.0:
        lim1 = lim1 + dt
        lim2 = lim2 + dt
        ax2.set_xlim(lim1, lim2)
        ax3.set_xlim(lim1, lim2)

    lpot.set_data([te, epot])
    lkint.set_data([te, ekint])
    ltot.set_data([te, etot])
    lang.set_data([te, am])
    
    trace.set_data(xt, yt)
    avector.set_offsets(np.array([r1[-1, 0], r1[-1, 1]]))
    fvector.set_offsets(np.array([r1[-1, 0], r1[-1, 1]]))
    avector.set_UVC(a[0], a[1])
    fvector.set_UVC(v[0], v[1])
    
    title.set_text("v(t) = %-5.3f" % norm(v))

    return (earth, avector, fvector, trace, lpot, lang)

# create animation for the matplotlib figure
anim = FuncAnimation(fig, update, frames = 20000, interval = 10,
                    init_func=init, blit=True, repeat = True)


def onClick(event):
    """callback function for the play and pause button"""
    global init_time
    if button_pause.description == "Pause":
        button_pause.description = "Play"

        anim.event_source.stop()
    else:
        button_pause.description = "Pause"
        init_time=datetime.now()

        anim.event_source.start()
        

def clear_data(b):
    """clear function for the Clear trace button"""
    global xt, yt
    
    xt = [];
    yt = [];

button_pause = Button(description="Play")
button_pause.on_click(onClick)
button_clear = Button(description="Clear trace")
button_clear.on_click(clear_data)

plt.show()

def on_dt_change(c):
    """callback for the change event of the slider"""
    global dt, r1, xt, yt
    
    anim.event_source.stop()
    
    dt = w_dt.value*0.01
    r1 = np.array([[10, 0],[10.0, 3.0*dt]])
    
    xt = []
    yt = []
    
    anim.event_source.start()

# the slider to change the time step
w_dt = IntSlider(value = 1, min = 1, max = 50, step=1, description="$\Delta t$: ", orientation='horizontal',
                layout = Layout(width="400px"))
w_dt.observe(on_dt_change, names='value')

anim.pause()
display(output, w_dt, HBox([button_pause, button_clear]))

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

## Legend
(How to use the interactive visualization)

### Interactive figures
The left figure shows the sun and earth system in two dimensions. The 
sun is in the center of the figure as shown in the yellow color. The 
earth is represented as a red dot. The trace of the earth's orbit is 
shown in the plot as the black line. The directions of the velocity 
and acceleration of the earth are shown as two vectors. The lengths
of the vectors indicate the values of the velocity and acceleration.

The top right figure shows how the energies change with time. The total 
energy is a summation of kinetic energy and potential energy. The plot 
is used to monitor the energy conservation of the system.

The bottom right figure shows angular momentum as a function of time.

### Controls
You can click the "Play & Pause" button to run or stop the simulation.
The slider is used to choose the time step $\Delta t$. The "Clear trace"
button can be used to remove the trace of the earth's orbit.